# Convolutional Neural Networks
## Cats and Dogs Face Recognition
---
In this notebook, we are going to train a **CNN** to classify cats and dogs images from a open database.

We will be using Pytorch as our framework.

The images in this database are color images that fall into one of two classes: Cat or Dog. A example is pictured below.

<img src='data/train/dog/dog.4051.jpg' width=20% height=20% />

In [1]:
import matplotlib.pyplot as plt

import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
import datetime

In [2]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print("CUDA isn't available.  Training on CPU ... :(")
else:
    print('CUDA is available!!  Training on GPU ... :)')

CUDA is available!!  Training on GPU ... :)


In [3]:
datapath = '../cnn_CatDog/data/'
batchSize = 25

# Defining transforms for the training data and testing:
trainTransforms = transforms.Compose([transforms.RandomRotation(30),
                                       #transforms.Resize((92,92)),
                                       transforms.RandomResizedCrop(92),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                            [0.5, 0.5, 0.5],
                                            [0.5, 0.5, 0.5])])
                                    

testTransforms = transforms.Compose([transforms.RandomRotation(30),
                                     transforms.Resize((92,92)),
                                     #transforms.RandomResizedCrop(crop),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
                                     

# Passing transforms:
trainData = datasets.ImageFolder(datapath + '/train', transform=trainTransforms)
testData = datasets.ImageFolder(datapath + '/test', transform=testTransforms)

# Separando test and validation data.
validSize = 0.15 # 20% do meu test vou dedicar para validação.

# Getting do value of trainData.
trainDataSize = len(trainData)
print(f'Size of Total Training Set: {trainDataSize}')
indices = list(range(trainDataSize))
#Aleatorizando os valores do train size.
np.random.shuffle(indices)

# Pegando um valor inteiro aprox. 20% do trainingset: 
split = int(np.floor(validSize * trainDataSize)) 
print(f'Split {split}')

# Separando o trainingset em train and valid using slice.
train_idx, valid_idx = indices[split:], indices[:split]
print(f'Size of Train Set: {len(train_idx)}\nSize of validation set: {len(valid_idx)}')

# define samplers for obtaining training and validation batches
trainSampler = SubsetRandomSampler(train_idx)
validSampler = SubsetRandomSampler(valid_idx)

# Carregando Train, Validation and Test data.                                
trainLoader = torch.utils.data.DataLoader(trainData, batch_size=batchSize, sampler=trainSampler)
validLoader = torch.utils.data.DataLoader(trainData, batch_size=batchSize, sampler=validSampler)
testLoader = torch.utils.data.DataLoader(testData, batch_size=batchSize)
print(f'Size of Test Set: {len(testLoader)*batchSize}')

classes = ['Cat', 'Dog']

Size of Total Training Set: 22500
Split 3375
Size of Train Set: 19125
Size of validation set: 3375
Size of Test Set: 2500


### Just verifying the the Sets

In [ ]:
test  = iter(trainLoader)
x, y = test.next()
print(x.shape)
print(y.shape)

### Show the TestSet images

In [ ]:
def imshow(img):
    #img = img / 2  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image

In [ ]:
# obtain one batch of training images
dataiter = iter(trainLoader)
images, labels = dataiter.next()
print(labels)

images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))

for idx in np.arange(5):
    ax = fig.add_subplot(1, 5, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])


In [ ]:
rgb_img = np.squeeze(images[3])
channels = ['red channel', 'g...reen channel', 'blue channel']

fig = plt.figure(figsize = (36, 36)) 
for idx in np.arange(rgb_img.shape[0]):
    ax = fig.add_subplot(1, 3, idx + 1)
    img = rgb_img[idx]
    ax.imshow(img, cmap='gray')
    ax.set_title(channels[idx])
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            val = round(img[x][y],2) if img[x][y] !=0 else 0
            ax.annotate(str(val), xy=(y,x),
                    horizontalalignment='center',
                    verticalalignment='center', size=8,
                    color='white' if img[x][y]<thresh else 'black')

### Specifying The Architechture

In [4]:
import torch.nn as nn
import torch.nn.functional as F

input_vector = 128*23*23

# define the CNN architecture
class Rob(nn.Module):
    def __init__(self):
        super(Rob, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 128, 3, padding=1)
        
        # MLP layers
        self.fc1 = nn.Linear(input_vector, 256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,32)
        self.fc4 = nn.Linear(32,2)

        # max pooling layer and dropout.
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.20)
        
    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        
        # Flattening image to MPL
        x = x.view(-1, input_vector)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x

# create a complete CNN
model = Rob()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

Rob(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=67712, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=2, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
)


### Specify Loss Function and Optimizer

In [5]:
import torch.optim as optim

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

### Training the Dataset

In [6]:
# number of epochs to train the model
n_epochs = 40 # you may increase this number to train a final model

valid_loss_min = np.Inf # track change in validation loss
t_loss = []
v_loss = []

start = datetime.datetime.now()
print(start)

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    print(f'\nStarting Training epoch: {epoch}', end=' / ')
    model.train()
    for data, target in trainLoader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    print(f'Starting Validation epoch: {epoch}')
    model.eval()
    for data, target in validLoader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(trainLoader.dataset)
    t_loss.append(train_loss)
    
    valid_loss = valid_loss/len(validLoader.dataset)
    v_loss.append(valid_loss)
        
    # print training/validation statistics 
    print(f'\tTraining Loss: {train_loss:.6f} \tValidation Loss: {valid_loss:.6f}')
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), 'model_CNN_catDog.pt')
        valid_loss_min = valid_loss
        
else:
#    print(output)
#    print(target)
    print(f'Training Loss: {t_loss}')
    print(f'Validation Loss: {v_loss}')
    final = datetime.datetime.now()

2020-01-08 19:25:42.309477

Starting Training epoch: 1 / Starting Validation epoch: 1
	Training Loss: 0.589684 	Validation Loss: 0.103923
Validation loss decreased (inf --> 0.103923). Saving model ...

Starting Training epoch: 2 / Starting Validation epoch: 2
	Training Loss: 0.588599 	Validation Loss: 0.103809
Validation loss decreased (0.103923 --> 0.103809). Saving model ...

Starting Training epoch: 3 / Starting Validation epoch: 3
	Training Loss: 0.587648 	Validation Loss: 0.103504
Validation loss decreased (0.103809 --> 0.103504). Saving model ...

Starting Training epoch: 4 / Starting Validation epoch: 4
	Training Loss: 0.584669 	Validation Loss: 0.102752
Validation loss decreased (0.103504 --> 0.102752). Saving model ...

Starting Training epoch: 5 / Starting Validation epoch: 5
	Training Loss: 0.578430 	Validation Loss: 0.101450
Validation loss decreased (0.102752 --> 0.101450). Saving model ...

Starting Training epoch: 6 / Starting Validation epoch: 6
	Training Loss: 0.570532

In [7]:
time_total = final-start
print(f'Total time on training: {time_total}')
print(f'Time per epoch: {time_total/n_epochs}')

Total time on training: 1:07:59.277744
Time per epoch: 0:01:41.981944


### Open the Better model

In [8]:
model.load_state_dict(torch.load('model_CNN_catDog.pt'))

<All keys matched successfully>

### Testing the model

In [9]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

model.eval()
# iterate over test data
for data, target in testLoader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    #print(output)
    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batchSize):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(testLoader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))


for i in range(len(classes)):
    if class_total[i] > 0:
        print(f'''Test Accuracy of {classes[i]}: {int(100*(class_correct[i]/class_total[i]))}%, 
              {np.sum(class_correct[i]), np.sum(class_total[i])}''')
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print(f'''\nTest Accuracy (Overall): \
      {100. * np.sum(class_correct) / np.sum(class_total)}%
      ({np.sum(class_correct)}/{np.sum(class_total)})''')
#30 epochs

Test Loss: 0.582237

Test Accuracy of Cat: 80%, 
              (1003.0, 1250.0)
Test Accuracy of Dog: 63%, 
              (788.0, 1250.0)

Test Accuracy (Overall):       71.64%
      (1791.0/2500.0)
